In [1]:
# import pdb; pdb.set_trace()
# import pudb; pudb.set_trace()

# df.style.background_gradient(
#     cmap='coolwarm', 
#     axis=None, 
#     vmin=-1, 
#     vmax=1
# )  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG'

# df.style.set_sticky(axis="index")

# pd.set_option('precision', 2)

# from pandas_profiling import ProfileReport
# profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)

# .plot_bokeh(sizing_mode="scale_height")
# .plot_bokeh(kind='hist', bins=1000, sizing_mode="scale_height", xlim=[0, .1])

# Imports

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 99)

import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

import datetime as dt
import pickle
import numpy as np
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_file("tmp/bokeh_output.html")
import ray

from data_layer import storage_adaptor, fsspec_factory, data_access, arrow_dataset
from tick_filter import streaming_tick_filter
from tick_sampler import streaming_tick_sampler, daily_stats
from utilities import date_fu, project_globals as g
from workflows import sampler_task, sampler_flow, syncbar_flow, syncbar_task
from workflows.configs import renko_v1, renko_v2, renko_v3
from sample_features import stacked
from filters import jma

In [3]:
config = renko_v2.config

prefix_data = '/data/trades'

prefix_meta = f"/bars/{config['meta']['config_id']}/meta"

prefix_df = f"/bars/{config['meta']['config_id']}/df"

prefix_sb = f"/bars/{config['meta']['config_id']}/sync_bars/clock_symbol=MUX"

In [4]:
df = data_access.fetch_sd_data('AEM', prefix=prefix_df, date='2019-01-03')

In [6]:
df

,bar_trigger,open_at,close_at,duration_td,tick_count,volume,dollars,tick_imbalance,volume_imbalance,price_high,price_low,price_open,price_close,price_range,price_return,price_vwap,price_wq05,price_wq25,price_wq50,price_wq75,price_wq95,label_side,label_outcome,label_rrr,label_outcome_at,label_start_at,label_end_at,symbol,date,config_id,renko_size
0,renko_up,2019-01-03 09:30:18.234388-05:00,2019-01-03 09:35:00.109437-05:00,0 days 00:04:41.875049,212,22335,909052.585701,-8,-3480,40.830002,40.619999,40.730000,40.830002,0.210003,0.100002,40.700810,40.650002,40.680000,40.700001,40.710098,40.770000,short,profit,-6.500000,2019-01-03 09:58:09.906826-05:00,2019-01-03 09:35:00.109437-05:00,2019-01-03 10:08:00.109437-05:00,AEM,2019-01-03,renko_v2,0.067411
1,renko_up,2019-01-03 09:35:00.109451-05:00,2019-01-03 09:37:08.829689-05:00,0 days 00:02:08.720238,140,20936,855564.189888,22,5566,40.980000,40.779999,40.840000,40.930000,0.200001,0.090000,40.865695,40.790001,40.840000,40.860001,40.900002,40.959999,short,profit,-5.000000,2019-01-03 09:58:08.527160-05:00,2019-01-03 09:37:08.829689-05:00,2019-01-03 10:10:08.829689-05:00,AEM,2019-01-03,renko_v2,0.067411
2,renko_down,2019-01-03 09:37:08.829762-05:00,2019-01-03 09:37:42.180288-05:00,0 days 00:00:33.350526,174,20603,841055.226219,-56,-4661,40.959999,40.770000,40.930000,40.770000,0.189999,-0.160000,40.821979,40.790001,40.799999,40.799999,40.825001,40.900002,neutral,neutral,0.000000,2019-01-03 09:40:23.358123-05:00,2019-01-03 09:37:42.180288-05:00,2019-01-03 10:10:42.180288-05:00,AEM,2019-01-03,renko_v2,0.067411
3,renko_down,2019-01-03 09:37:42.180980-05:00,2019-01-03 09:39:10.119634-05:00,0 days 00:01:27.938654,53,6375,259841.875458,-23,-2475,40.820000,40.700001,40.770000,40.700001,0.119999,-0.070000,40.759510,40.709999,40.740002,40.759998,40.779999,40.799999,long,profit,2.000000,2019-01-03 09:42:19.007017-05:00,2019-01-03 09:39:10.119634-05:00,2019-01-03 10:12:10.119634-05:00,AEM,2019-01-03,renko_v2,0.067411
4,renko_up,2019-01-03 09:39:10.119977-05:00,2019-01-03 09:42:19.007143-05:00,0 days 00:03:08.887166,203,34020,1385703.621960,-1,-8056,40.860001,40.669998,40.700001,40.840000,0.190002,0.139999,40.732029,40.669998,40.680000,40.700001,40.779999,40.840000,short,profit,-4.000000,2019-01-03 09:58:09.906826-05:00,2019-01-03 09:42:19.007143-05:00,2019-01-03 10:15:19.007143-05:00,AEM,2019-01-03,renko_v2,0.067411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,renko_up,2019-01-03 15:03:52.506671-05:00,2019-01-03 15:05:29.485415-05:00,0 days 00:01:36.978744,250,53349,2200828.145477,36,16937,41.279999,41.209999,41.220001,41.279999,0.070000,0.059998,41.253410,41.220001,41.240002,41.257500,41.266499,41.279598,short,profit,-3.500000,2019-01-03 15:37:07.122726-05:00,2019-01-03 15:05:29.485415-05:00,2019-01-03 15:38:29.485415-05:00,AEM,2019-01-03,renko_v2,0.067411
46,renko_down,2019-01-03 15:05:29.487673-05:00,2019-01-03 15:12:44.070782-05:00,0 days 00:07:14.583109,329,83211,3433029.402451,-31,-33735,41.329399,41.139999,41.274200,41.139999,0.189400,-0.134201,41.256918,41.160000,41.220001,41.270000,41.305801,41.316002,long,profit,2.000000,2019-01-03 15:28:49.472242-05:00,2019-01-03 15:12:44.070782-05:00,2019-01-03 15:45:44.070782-05:00,AEM,2019-01-03,renko_v2,0.067411
47,renko_down,2019-01-03 15:12:49.557768-05:00,2019-01-03 15:37:07.122726-05:00,0 days 00:24:17.564958,957,109383,4503298.266266,-65,-12751,41.275101,41.060001,41.130001,41.060001,0.215099,-0.070000,41.170001,41.090000,41.130001,41.174999,41.200001,41.259998,long,profit,3.000000,2019-01-03 15:46:53.308957-05:00,2019-01-03 15:37:07.122726-05:00,2019-01-03 16:10:07.122726-05:00,AEM,2019-01-03,renko_v2,0.067411
48,renko_up,2019-01-03 15:37:07.122745-05:00,2019-01-03 15:43:41.339834-05:00,0 days 00:06:34.217089,429,43759,1799086.139980,31,5107,41.200001,41.049999,41.060001,41.200001,0.150002,0.139999,41.113511,41.070000,41.090000,41.099998,41.130001,41.180000,neutr

In [ ]:
sum(abs(df.label_rrr)>=3) / df.shape[0]

In [ ]:
abs(df.label_rrr).hist(bins=30)

# load multi-file dataset

In [ ]:
ds = arrow_dataset.get_dataset('/bars/renko_v2/df')

df = ds.to_table().to_pandas()

sdf = df[df.symbol=='SAND']

In [ ]:
ds = arrow_dataset.get_dataset('/bars/renko_v2/sync_bars')

df2 = ds.to_table().to_pandas()

sdf2 = df2[(df2.clock_symbol=='IAG') & (df2.symbol=='GLD')].reset_index(drop=True)

# Price gap fill

In [ ]:
fdf = stacked.fill_gaps_dates(sdf, 'price_vwap')

In [ ]:
fdf[['price_vwap','price_high','price_low']].plot_bokeh(sizing_mode="scale_height")

# JMA filter

In [ ]:
jma.jma_expanding_filter(fdf.price_vwap, winlen=200, power=1, phase=0)

In [ ]:
jma.jma_filter_df(fdf, 'price_vwap', winlen=205, power=1)

In [ ]:
jma.jma_filter_df(fdf, 'price_vwap', winlen=2000, power=1).price_vwap_jma.plot()

In [ ]:
jma.jma_filter_df(fdf, 'price_vwap', winlen=1000, power=1).price_vwap_jma.diff().hist(bins=40)

In [ ]:
fdf.date.value_counts().hist(bins=40)

# Ray setup

In [ ]:
ray.init(dashboard_host='0.0.0.0', dashboard_port=1111, ignore_reinit_error=True)

In [ ]:
ray.shutdown()

In [ ]:
all_symbols = data_access.list_sd_data(prefix='/bars/renko_v2/df', source='remote')

In [ ]:
large_caps, mid_caps = syncbar_task.segment_symbols(
    all_symbols, start_date='2019-01-01', end_date='2021-01-01'
)

In [ ]:
large_caps

In [ ]:
mid_caps

# sync bars

In [ ]:
config = {
    'config_id': 'renko_v2',
    'start_date': '2019-01-01',
    'end_date': '2021-01-01',
    'source': 'remote',
    'destination': 'both',
    'on_ray': True,
}

In [ ]:
syncbar_flow.run(config)

# Tick Sampler 

In [ ]:
bds = sampler_flow.run(config)

# fracdiff test

In [ ]:
import statsmodels.tsa.stattools as stattools
import fracdiff

X = bdf.price_close.values.reshape(-1, 1)

fs = fracdiff.FracdiffStat(mode="valid")

Xdiff = fs.fit_transform(X)

_, pvalue, _, _, _, _ = stattools.adfuller(Xdiff)

corr = np.corrcoef(X[-Xdiff.size :, 0], Xdiff.reshape(-1))[0][1]

print("* Order: {:.2f}".format(fs.d_[0]))
print("* ADF p-value: {:.2f} %".format(100 * pvalue))
print("* Correlation with the original time-series: {:.2f}".format(corr))

spx = bdf.price_close

import matplotlib.pyplot as plt

spx_diff = pd.Series(Xdiff.reshape(-1), index=spx.index[-Xdiff.size :])

fig, ax_s = plt.subplots(figsize=(24, 6))
plt.title("S&P 500 and its differentiation preserving memory")
ax_d = ax_s.twinx()

plot_s = ax_s.plot(spx, color="blue", linewidth=4, label="S&P 500 (left)")
plot_d = ax_d.plot(
    spx_diff,
    color="orange",
    linewidth=4,
    label=f"S&P 500, {fs.d_[0]:.2f} th diff (right)",
)
plots = plot_s + plot_d

ax_s.legend(plots, [p.get_label() for p in plots], loc=0)
plt.show()